In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import periodogram, medfilt2d
from scipy.fftpack import next_fast_len

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
figsize=[9.5 , 5]

In [ ]:
maxFreq = 100
maxHarmonic = 25
peddingFactor = 100
detectionThresh = 2

# consider using the axial direction

In [ ]:
t = pd.read_csv('Hwp - motor - bearing 1- 12_16_16 - 18_12.csv').iloc[:, 0].values
x = pd.concat([
    pd.read_csv('Hwp - motor - bearing 1- 12_16_16 - 18_12.csv').iloc[:, 2:],
    pd.read_csv('Hwp - motor - bearing 2- 12_16_16 - 18_12.csv').iloc[:, 2:],
              ], axis=1).values
x

In [ ]:
nfft = next_fast_len(x.shape[0]) * peddingFactor
f, pxx = periodogram(x, fs=1 / t[1], return_onesided=True, axis=0, nfft=nfft)

In [ ]:
cond = f < maxFreq * maxHarmonic
pxx = pxx[cond]
f = f[cond]

In [ ]:
# here the computation time can be reduced by decimation
winSize = (np.round(10/ f[1] / 2) * 2).astype(int) + 1
pxxMed = medfilt2d(np.log(pxx), (winSize, 1))

In [ ]:
for c in range(pxx.shape[1]):
    plt.figure(figsize=figsize)
    cond = (f < maxFreq) & (f > 5)
    plt.plot(f[cond], np.log(pxx[cond, c]))
    plt.plot(f[cond], pxxMed[cond, c])
    plt.grid()
    plt.show()

In [ ]:
logPxxDetrand = np.log(pxx) - pxxMed
logPxxDetrand[logPxxDetrand < detectionThresh] = 0
logPxxDetrand += 1
probAxisProd = np.prod(logPxxDetrand, axis=1)

nSearch = (f < maxFreq).sum()
probHarmonicProd = np.ones((nSearch, ))

viewCond = (f[:nSearch] > 15) & (f[:nSearch] < 40)
    
for harmonic in range(1, maxHarmonic + 1):
    plt.figure(figsize=figsize)
    before = probHarmonicProd.copy()
    probHarmonicProd *= probAxisProd[:nSearch * harmonic:harmonic]
    plt.title("Harmonic No. {harmonic}".format(harmonic=harmonic))
    plt.semilogy(f[:nSearch][viewCond], probHarmonicProd[viewCond])
    plt.semilogy(f[:nSearch][viewCond], before[viewCond])
    plt.legend(['after', 'before'])
    plt.grid()
    plt.show()

In [ ]:
f[np.argmax(probHarmonicProd[f[:nSearch] < 50])]